In [1]:
import torch
from transformers import BertTokenizer, BertModel
from Code.model import *

# Data preprocessing

## How to load tokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained('./pytorch-ernie', do_lower_case = True)

## Data loader
pos:   372
neg:   4651
total: 5023


In [2]:
import torch
from transformers import BertTokenizer, BertModel
import glob
import chinese_converter
from string import punctuation
import ast
import numpy as np
import re

class AMLDataset(torch.utils.data.Dataset):
    def __init__(self, data_path='./data/training_set/', model_path='./pytorch-ernie'):
        self.paths = glob.glob(data_path+"*.txt")
#         self.paths = glob.glob(data_path+"108.txt")+glob.glob(data_path+"209.txt")+glob.glob(data_path+"1.txt")
        self.tokenizer = BertTokenizer.from_pretrained('./pytorch-ernie', do_lower_case = True)
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        with open(path, encoding = 'utf-8') as f:
            text = f.read().split('\n')
            target = " ".join(ast.literal_eval(text[0]))
            input_data = "".join(text[1:])
#             input_data = re.sub(r'[A-Za-z]','', input_data)
            # remove punctuation
#             punctuation_characters = punctuation+'。、·「」！，）：（【】'
#             input_data = input_data.translate(str.maketrans('', '',punctuation_characters ))
            # remove space
#             input_data = input_data.replace(' ','')

            # remove data without context
            if len(input_data) < 10:
                target = ''
        return input_data, target
        
def convert_batch_token_target(batch_input_data, batch_target):
    batch_input_data = [chinese_converter.to_simplified(d) for d in batch_input_data]
    input_tensor = tokenizer(batch_input_data, padding=True, return_tensors="pt", add_special_tokens=False)
    input_token, padding_mask = input_tensor["input_ids"], input_tensor['attention_mask']
    batch_target = process_target(input_token, batch_target)
    return  input_token, padding_mask, torch.tensor(batch_target)

# process target binary array
def process_target(input_token, batch_target):
    targets = []
    for slice_token, names in zip(input_token, batch_target):
#         text = tokenizer.decode(slice_token, clean_up_tokenization_spaces=False).split(' ')
        text = tokenizer.convert_ids_to_tokens(slice_token)
        target = [0]*len(text)
        for name in names.split(" "):
            name = chinese_converter.to_simplified(name)
            for i in range(len(text)-len(name)):
                if name == "".join(text[i:i+len(name)]):
                    target[i:i+len(name)] = [1]*len(name)
        targets.append(target)
    return targets

# map binary prediction to text
def map_batch_prediction_to_text(input_tokens, predictions):
    batch_text_prediction = []
    for slice_token, prediction in zip(input_tokens, predictions):
#         text = tokenizer.decode(slice_token, clean_up_tokenization_spaces=True).split(' ')
        text = tokenizer.convert_ids_to_tokens(slice_token)
        text_prediction = []
        name = ''
        for t, p in zip(text, prediction):
            if p==0:
                if name != '':
                    text_prediction.append(name)
                name = ''
            else:
                if t != '[PAD]':
                    name += t
        batch_text_prediction.append(text_prediction)
    return [list(set(p)) for p in batch_text_prediction]

def split_data(token_tensor, mask_tensor, target_tensor, max_length):
    bs, seq = token_tensor.size()
    output_token_tensor = []
    output_mask_tensor = []
    output_target_tensor = []
    for i in range(int(np.ceil(seq/max_length))):
        output_token_tensor.append(token_tensor[:,i*max_length:(i+1)*max_length])
        output_mask_tensor.append(mask_tensor[:,i*max_length:(i+1)*max_length])
        output_target_tensor.append(target_tensor[:,i*max_length:(i+1)*max_length])
    return output_token_tensor, output_mask_tensor, output_target_tensor

In [3]:
from pytorch_lightning.metrics.sklearns import F1
metric = F1()

In [5]:
data = AMLDataset()
dataloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=1)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BertClassifier().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for idx, (d, t) in enumerate(dataloader):
    print('=={}=='.format(idx))
    d, m, t = convert_batch_token_target(list(d), list(t))
    print(d.size(), m.size(), t.size())
    split_d, split_m, split_t = split_data(d, m, t, 512)
    for (d, m), t in zip(zip(split_d, split_m), split_t):
        optimizer.zero_grad()
        output = model(d.to(device))
        loss = mask_BCE_loss(y_pred = output.float().to(device), 
                             y_true = t.float().to(device), 
                             padding_mask = m.float().to(device), 
                             weight = 500)
        print("loss:", loss.item())
        print("F1 score:", metric(output.float().round(), t.float().round()).item())
        loss.backward()
        prediction = map_batch_prediction_to_text(d, output.round())
        print("map_batch_prediction_to_text:",prediction)
        optimizer.step()
    
    

==0==
torch.Size([4, 750]) torch.Size([4, 750]) torch.Size([4, 750])
loss: 3.1995902061462402
F1 score: 0.01640624925494194


/Users/ansenhuang/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


map_batch_prediction_to_text: [[], ['大角嘴私家车路中心起火冒烟[UNK]10:32[UNK]凌晨2时许，大角嘴福全街58号洪圣庙对开，一辆停于路中心的私家车起火冒烟，有途人目击司机下车逃离现', '，火势波及附近4辆电单车，私家车车尾被焚毁，消防到场开喉灌救，经初步调查，相信起火原因有'], ['印制厂印制是以印制新台币来看待，不但要品质好，而且还要防止有任何的资安疑虑。苏贞昌指出', '有关新的数位身分证，相信中央印制厂会为全体国人做最好的服务，政府也这样要求。内政部规划明年10月起换发数位身分识别证（newei##d），镜周刊报导', '苏贞昌：数位身分证重视资安对于数位身分证采限制性招标委托中央印制厂印制，遭外界质疑有图利特定厂商之嫌，行政院长苏贞昌今天表示，新的身分证非常重要，就像新台币一样，政府把身分证交给中'], []]
loss: 0.2944474220275879
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], ['检方不服提抗告，最高法']]
==1==
torch.Size([4, 950]) torch.Size([4, 950]) torch.Size([4, 950])
loss: 0.9617899656295776
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.5443954467773438
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], ['指考国文出题呈现跨领域、跨学科特色，第8题关于蜜蜂的两种舞步分析；第27至29题成语动物学乌鸦生态；第6题德国旅行家；第41至42题社会学是什么；同时考题也呈现生活情境化，像是第17到19题取材周芬伶的[UNK]说瓷[UNK]，文中讨论的是关于瓷器的历史沿革，与喝茶所引发的美感。张青松也提到，考题另一', '张青松认为，今', '特色', '的台大中文', '第9题的杂文《重披白袍》，都是不错的文选。中正高中国文教', '教授柯庆明[UNK]中国文学之美的价值性[UNK]，聚焦于[UNK]鸿门宴[UNK]，谈《史记》的人物性格之美。建国

/Users/ansenhuang/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


map_batch_prediction_to_text: [[], [], ['需', '员'], []]
loss: 0.13971452414989471
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.1320527344942093
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.12494015693664551
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.11599859595298767
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
==4==
torch.Size([4, 822]) torch.Size([4, 822]) torch.Size([4, 822])
loss: 6.90348482131958
F1 score: 0.001953125
map_batch_prediction_to_text: [['云', '大', '气'], ['少', ':', '独', '万能科大工业管理', '会', '动', '4.', '宏大会', '立', '少杰', '事务', '生', '旭辰会', '选5', '独立', '凯', '1', '登自动化', '股6', '[UNK]', '执', '2', '怡铭', '事', '国', '股独', '因:辞职', '任', '人'], ['审', '万', '2000', '与', '文', '渔'], []]
loss: 0.11238963156938553
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
==5==
torch.Size([4, 2519]) torch.Size([4, 2519]) torch.Size([4, 2519])
loss: 0.4867265224456787
F1 score: 0.0
map_batch_prediction_to

...

loss: 0.09698410332202911
F1 score: 0.0
map_batch_prediction_to_text: [[], ['郑'], ['祺', '咚', '柏均', '碧'], ['东城', '陈建州', '范玮琪', '高以翔', '毛加恩', '汪东城', '王钧', '恩', '邱秀珠']]
loss: 0.01519249938428402
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['郭柏均'], []]
loss: 0.08951146900653839
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['咚咚', '碧', '郭柏均', '林咚咚'], []]
loss: 0.05029263719916344
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['林咚咚', '碧', '郭柏均'], []]
loss: 0.020225059241056442
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['郭柏均', '郭孟雍'], []]
==1171==
torch.Size([4, 992]) torch.Size([4, 992]) torch.Size([4, 992])
loss: 0.08729119598865509
F1 score: 0.0
map_batch_prediction_to_text: [['黄升勇', '周倪安'], [], [], ['孙正义']]
loss: 0.010477183386683464
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], ['孙正义', '正义']]
==1172==
torch.Size([4, 944]) torch.Size([4, 944]) torch.Size([4, 944])
loss: 0.005150793120265007
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [

loss: 0.06035306304693222
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['董景翔', '唐玺'], ['范', '张国光', '侯博明']]
loss: 0.025280563160777092
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], ['侯博明']]
==1191==
torch.Size([4, 2934]) torch.Size([4, 2934]) torch.Size([4, 2934])
loss: 0.11133334040641785
F1 score: 0.0403645820915699
map_batch_prediction_to_text: [['臣'], ['。', '太平', '，', '缪竹怡', '。缪竹怡', '姚介修', '缪嫌', '缪员', '长缪竹怡'], [], ['侯孝贤', '杨德昌', '蔡明亮']]
loss: 0.01264948770403862
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.0028116977773606777
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.002523926319554448
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.0023852032609283924
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.002404423663392663
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
==1192==
torch.Size([4, 1308]) torch.Size([4, 1308]) torch.Size([4, 1308])
loss: 0.1979312002658

loss: 0.004935781937092543
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
==1209==
torch.Size([4, 2208]) torch.Size([4, 2208]) torch.Size([4, 2208])
loss: 0.06290099769830704
F1 score: 0.005859375
map_batch_prediction_to_text: [['吴', '吴丽玲', '陈慰'], [], ['吴汉章'], []]
loss: 0.006882466375827789
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.0062043252401053905
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['吴汉章'], []]
loss: 0.002231612103059888
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.002197773428633809
F1 score: 0.0
map_batch_prediction_to_text: [[], [], [], []]
==1210==
torch.Size([4, 3556]) torch.Size([4, 3556]) torch.Size([4, 3556])
loss: 0.1839161068201065
F1 score: 0.0338541679084301
map_batch_prediction_to_text: [[], [], [], ['黄子窈', '蔡丰益', '夏子茵', '李金龙', '亿圆富', '圆富', '陈进村']]
loss: 0.7629488706588745
F1 score: 0.03125
map_batch_prediction_to_text: [[], [], [], []]
loss: 0.00219598226249218
F1 score: 0.0
map_batch_p

map_batch_prediction_to_text: [[], ['杨宗斌'], [], []]
==1226==
torch.Size([4, 3188]) torch.Size([4, 3188]) torch.Size([4, 3188])
loss: 0.13539503514766693
F1 score: 0.0
map_batch_prediction_to_text: [['份', '司董', '股', '英凯禾荣产', '陈英凯禾荣产'], ['汤佳', '雄', '峰'], ['海', '鸿', '谷口英男', '方荣熙', '鸿海'], ['李裕华先生']]
loss: 0.08053714036941528
F1 score: 0.0
map_batch_prediction_to_text: [[], ['雄'], ['海', '戴正吴', '志圣', '郭台铭视', '郭台铭', '方荣熙', '鸿海'], []]
loss: 0.048036787658929825
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['郭台铭', '海', '郭董', '鸿海'], []]
loss: 0.1540527194738388
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['柯文哲', '柯', '夏', '戴正吴', '夏普', '王金平', '鸿夏', '郭台铭', '王连', '郭柯王', '韩国瑜', '郭董', '郭办', '鸿海'], []]
loss: 0.11156616359949112
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['黄秋莲', '夏', '郭', '夏普', '戴正吴', '郭台铭', '钱妈妈', '郭董', '鸿海'], []]
loss: 0.06789539009332657
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['康', '海', '刘宥彤', '郭台铭', '王志超', '仁宝', '郭董', '奇美', '鸿海'], []]
loss: 

map_batch_prediction_to_text: [['叶'], [], [], []]
==1242==
torch.Size([4, 2294]) torch.Size([4, 2294]) torch.Size([4, 2294])
loss: 0.06845168769359589
F1 score: 0.0
map_batch_prediction_to_text: [['李纲', '林右昌', '黄骏逸', '林', '曾姿雯', '曾处长'], [], ['何明喧'], ['政杰']]
loss: 0.04513305053114891
F1 score: 0.0
map_batch_prediction_to_text: [['曾姿雯', '黄骏逸'], ['郭冠缨', '侯建业'], ['何明喧', '林煜腾', '明喧'], []]
loss: 0.01881570741534233
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['苏贞昌', '林绿红'], []]
loss: 0.028509464114904404
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['庭瑞', '林绿红', '庄庭瑞'], []]
loss: 0.021011939272284508
F1 score: 0.0
map_batch_prediction_to_text: [[], [], ['邱伊翎'], []]
==1243==
torch.Size([4, 1383]) torch.Size([4, 1383]) torch.Size([4, 1383])
loss: 0.019428648054599762
F1 score: 0.0
map_batch_prediction_to_text: [[], ['林幸蓉'], ['邱正弘'], []]
loss: 0.055967316031455994
F1 score: 0.0
map_batch_prediction_to_text: [['美元', '护用', 'youtube早', '美', 'youtube没', '是同类案件', '联邦', '儿童资', '评', '